# Does Enhancing Images also Enhance the Outcome of Image Recognition of Neural Networks

# General Layout for this program

1. Imports
2. Load the training set
3. Load in the test set
4. Load in the test set labels
5. Create a copy of the image set and apply CLAHE to it
6. Load in 2 pretrained resnets
7. Train resnet1 on OG images, train resnet2 on CLAHE images
8. Compare performance of both

# Imports

In [68]:
import torch
from torchvision import *
from torch import optim
import torch.nn as nn
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
import os
from PIL import Image
from timeit import default_timer as timer
import cv2
from torch.utils.data import Dataset, DataLoader


# Organize ImageNet so there are 50 images from each catagory in a test folder

-> All under imageNet/ILSVRC/Data/CLS-LOC/test

In [69]:
import shutil

headDirectory = './imageNet/ILSVRC/Data/CLS-LOC/'

if os.path.isdir('./imageNet/ILSVRC/Data/CLS-LOC/' + 'test'):
    shutil.rmtree(headDirectory + 'test')
    
os.mkdir(headDirectory + 'test')

for folder in sorted(os.listdir(headDirectory + 'train')):

    if (folder == 'sals' or folder == '.DS_Store'):
        continue
    
    f = os.path.join(headDirectory + 'train/', folder)
    os.mkdir(headDirectory + 'test/' + folder)
    counter = 0

    for file in sorted( os.listdir(f)):
        #print(file)
        if counter == 50:
            break
        else:
            shutil.copy(f + '/' + file, headDirectory + 'test/' + folder)
            counter += 1

KeyboardInterrupt: 

# Import the Spotted Salamander image set


Get the size fo the largest and smallest images in the spotted salamander set

In [ ]:
smallestHeight = 1000
smallestWidth = 1000
largestHeight = 0
largestWidth = 0
numImages = 0

directory = './imageNet/ILSVRC/Data/CLS-LOC/train/n01632458'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        numImages += 1
        img = cv2.imread(f)

        if img.shape[0] < smallestHeight:
            smallestHeight = img.shape[0]
        if img.shape[1] < smallestWidth:
            smallestWidth = img.shape[1]
        if img.shape[0] > largestHeight:
            largestHeight = img.shape[0]
        if img.shape[1] > largestWidth:
            largestWidth = img.shape[1]

print("Number of Images: " + str(numImages))
# print('Smallest Height = ' + str(smallestHeight))
# print('Smallest Width = ' + str(smallestWidth))
# print('Largest Height = ' + str(largestHeight))
# print('Largest Width = ' + str(largestWidth))

Number of Images: 1150


# Load the images in

In [ ]:
import torchvision
import numpy
from torchvision import transforms
transforms = transforms.Compose(
[
   transforms.ToTensor(),
   transforms.Resize((smallestHeight, smallestWidth)),
   transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
])

directory = 'imageNet/ILSVRC/Data/CLS-LOC/train'
train_dataset = torchvision.datasets.ImageFolder(directory, transforms)
num_classes = len(train_dataset.classes)
print('num classes = ', len(train_dataset.classes))
print(type(train_dataset.classes))

print(train_dataset.classes[0])

# Need to remove first 50 images from each class
#train_dataset = torch.utils.data.Subset(train_dataset, range(50, len(train_dataset)))


print('len = ', train_dataset.__len__())
print('item = ', train_dataset.__getitem__(0))

print(type(train_dataset))
print(type([train_dataset[0]]))
print(type([train_dataset[0][0]]))

num classes =  1001
<class 'list'>
n01440764
len =  1132059
item =  (tensor([[[0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         ...,
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764]],

        [[0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         ...,
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764]],

        [[0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764, 0.8764],
         [0.8764, 0.8764, 0.8764,  ..., 0.8764, 0.8764

In [ ]:
#Doesnt show images right now
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

Turn the loaded images into a dataloader datatype

In [ ]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Create the learning model

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

print(input)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()

tensor([[ 0.3797, -1.3058,  2.1702,  1.1050,  0.6462],
        [ 0.8842,  0.2101, -1.9627,  1.5388,  0.0451],
        [ 0.1291, -0.9840, -0.0348, -0.7895, -0.7323]], requires_grad=True)
tensor([2, 4, 0])


In [ ]:
resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT, num_classes=num_classes - 1)

criterion = nn.CrossEntropyLoss()

print(resnet.parameters())

optimizer = optim.SGD(resnet.parameters(), lr=0.0001, momentum=0.9)

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 128)
device = 'cpu'


<generator object Module.parameters at 0x2cba6c190>


Train the model

In [ ]:
from cgitb import reset


n_epochs = 5
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

resnet.train()

for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    batch_idx = -1
    for i, (data_, target_) in enumerate(train_loader):
        batch_idx += 1


        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        
        outputs = resnet(data_)
        print("len targets", len(target_))
        print("len outputs", len(outputs))

        print("target_", target_)
        print("outputs", outputs)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t = 0
    correct_t = 0
    with torch.no_grad():
        resnet.eval()
        for data_t, target_t in (train_loader):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = resnet(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(train_loader))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')

        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(resnet.state_dict(), 'resnet.pt')
            print('Improvement-Detected, save-model')
    resnet.train()

Epoch 1

len targets 100
len outputs 100
target_ tensor([333, 934, 485, 266, 709,   7, 706, 355, 331, 263, 675, 616, 987, 606,
        664, 624, 520, 908, 350, 828, 799, 277, 955, 467, 712, 280, 877, 990,
        768, 804, 471, 692, 219, 518, 388, 388, 267, 290, 732, 308,  49, 237,
        184, 839, 263, 341, 879, 287, 944, 594, 662, 512,  40, 633, 450, 977,
        283, 480, 896, 842, 943, 815, 788, 291, 759, 966, 450, 646, 931, 799,
        542, 954, 114,   0,  26, 945,  80, 648, 858, 390, 842, 892, 968, 203,
        381, 834,  45, 604, 362, 425, 886, 608,  62, 769, 195,  72, 176, 705,
        399, 692])
outputs tensor([[-1.0648, -0.5896, -0.6819,  ..., -0.3720,  0.2740, -0.3472],
        [-0.0666, -0.8909,  0.2444,  ...,  0.1824,  0.3586,  0.5651],
        [ 0.1070,  0.8505,  0.0860,  ..., -1.1745,  1.6643,  0.1102],
        ...,
        [-0.6421, -1.1351,  1.2851,  ..., -0.2563,  0.6475, -0.5935],
        [-0.9007, -1.8831,  1.2034,  ..., -0.5255,  0.5767,  0.6071],
        [-0.376

IndexError: Target 333 is out of bounds.

## Save model

In [ ]:
save_path = 'SalamanderResnet.pth'
torch.save(resnet.state_dict(), save_path)

# Switch into eval mode and test it on some images

In [ ]:
from cgi import test

resnet.eval()

testSet = []
directory = './imageNet/ILSVRC/Data/CLS-LOC/test'


#Load in all test salamader images
testData = torchvision.datasets.ImageFolder(directory, transforms)
testData = torch.utils.data.Subset(testData, range(50))

print(type(testData))
print(len(testData))

<class 'torch.utils.data.dataset.Subset'>
50


In [ ]:
#Switch Model into eval mode
resnet.eval()
test_loader = DataLoader(testData, batch_size=50, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
with torch.no_grad():
    running_loss = 0.
    running_corrects = 0
    for i, (inputs, labels) in enumerate(test_loader):
        print("label = ", labels)
        print("inputs = ", inputs)
        print("loop " + str(i))
        if (i == 50):
            break
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = resnet(inputs)
        print('outputs ', outputs )
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        if i == 0:
            print('======>RESULTS<======')
            #images = torchvision.utils.make_grid(inputs[:4])
    epoch_loss = running_loss / len(testData)
    epoch_acc = running_corrects / len(testData) * 100.
    print('[Test #{}] Loss: {:.4f} Acc: {:.4f}%'.
          format(epoch, epoch_loss, epoch_acc))

label =  tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])
inputs =  tensor([[[[0.5744, 0.3828, 0.2692,  ..., 0.1600, 0.0672, 0.5370],
          [0.4968, 0.3041, 0.2234,  ..., 0.0667, 0.0786, 0.3704],
          [0.5122, 0.2530, 0.4054,  ..., 0.1577, 0.1067, 0.1071],
          ...,
          [0.2044, 0.3462, 0.3323,  ..., 0.3689, 0.2107, 0.2716],
          [0.2433, 0.5014, 0.2148,  ..., 0.2467, 0.1235, 0.0477],
          [0.5387, 0.4669, 0.3714,  ..., 0.3030, 0.2539, 0.1776]],

         [[0.6834, 0.5164, 0.2964,  ..., 0.2198, 0.2356, 0.6205],
          [0.5812, 0.3899, 0.2398,  ..., 0.1474, 0.1986, 0.4012],
          [0.6164, 0.3831, 0.5370,  ..., 0.2269, 0.1961, 0.1569],
          ...,
          [0.3078, 0.4447, 0.4244,  ..., 0.4391, 0.3629, 0.4134],
          [0.3622, 0.5387, 0.2696,  ..., 0.4215, 0.2722, 0.5032],
          [0.6350, 0.5920, 0.4177,  ..., 0.3695,